# Building ANN to predict the Churn Rate

In [1]:
# Importing the libraries

import numpy as np
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [2]:
# Encoding categorical data
# Encoding the Independent Variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

labelencoder_X_1 = LabelEncoder()
X[:,2] = labelencoder_X_1.fit_transform(X[:,2])

#labelencoder_X_2 = LabelEncoder()
#X[:,2] = labelencoder_X_2.fit_transform(X[:,2])

ct = ColumnTransformer([("Geography", OneHotEncoder(),[1])], remainder="passthrough")
X = np.array(ct.fit_transform(X), dtype=np.int64)
#avoiding dummy trap:
X = X[:, 1:]

In [3]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [4]:
#Part 2 : Making the ANN:

#Importing the Keras Libraries & Packages
from keras.models import Sequential
from keras.layers import Dense

#Initialize the ANN
classifier = Sequential()

#Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform',activation = 'relu', input_shape = (11,)))

#Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform',activation = 'relu'))

#Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform',activation = 'sigmoid'))

#Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics= ['accuracy'] )

#Fitting the ANN to the training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)


Using TensorFlow backend.


Epoch 1/100
8000/8000 [==============================] - 1s 123us/step - loss: 0.4888 - accuracy: 0.79580s - loss: 0.5132 - accura
Epoch 2/100
8000/8000 [==============================] - 1s 84us/step - loss: 0.4283 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 84us/step - loss: 0.4231 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 78us/step - loss: 0.4186 - accuracy: 0.8176
Epoch 5/100
8000/8000 [==============================] - 1s 78us/step - loss: 0.4165 - accuracy: 0.8271
Epoch 6/100
8000/8000 [==============================] - 1s 79us/step - loss: 0.4145 - accuracy: 0.8313
Epoch 7/100
8000/8000 [==============================] - 1s 80us/step - loss: 0.4132 - accuracy: 0.8306
Epoch 8/100
8000/8000 [==============================] - 1s 79us/step - loss: 0.4117 - accuracy: 0.8321
Epoch 9/100
8000/8000 [==============================] - 1s 80us/step - loss: 0.4106 - accuracy: 0.8326
Epoch 10/100
8000/8000 [=============

8000/8000 [==============================] - 1s 85us/step - loss: 0.4003 - accuracy: 0.8341
Epoch 78/100
8000/8000 [==============================] - 1s 83us/step - loss: 0.4006 - accuracy: 0.8332
Epoch 79/100
8000/8000 [==============================] - 1s 83us/step - loss: 0.4006 - accuracy: 0.8360
Epoch 80/100
8000/8000 [==============================] - 1s 84us/step - loss: 0.4006 - accuracy: 0.8351
Epoch 81/100
8000/8000 [==============================] - 1s 86us/step - loss: 0.4002 - accuracy: 0.8345
Epoch 82/100
8000/8000 [==============================] - 1s 84us/step - loss: 0.4006 - accuracy: 0.8365
Epoch 83/100
8000/8000 [==============================] - 1s 85us/step - loss: 0.4003 - accuracy: 0.8331
Epoch 84/100
8000/8000 [==============================] - 1s 83us/step - loss: 0.4007 - accuracy: 0.8347 0s - loss: 0.3992 - accura
Epoch 85/100
8000/8000 [==============================] - 1s 83us/step - loss: 0.4007 - accuracy: 0.8351
Epoch 86/100
8000/8000 [=================

In [5]:
#Part - 3 : Making the prediction and evaluating the model

#Predicting the test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred> 0.5)  #If y_pred>0.5 then true(1) else False(0)
print(y_pred)

#Making the confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test,y_pred)
acc = accuracy_score(y_pred,y_test)
print(acc)

#To predict if the customer will stay with the bank or not
#Below are the given data
'''
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000
'''
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))) #Classifier need an array of data hence used np.array(). The data we provided is not scaled hence we used sc.tranform to scale it in the same as the model was trained before. 
#We have used two square bracets in the np.array. single [] = Vertical vector(column) double [[]]= Horizontal vector, i.e row
new_prediction = (new_prediction> 0.5) #Again we need to classify the result as true or false

print('Customer Leaves Bank?:', new_prediction[0][0])

[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
0.8395
Customer Leaves Bank?: False


In [14]:
#Part 4

# Evaluating, Improving and Tuning the ANN
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation='relu', input_shape=(11,)))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation='relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics= ['accuracy'] )
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100 )
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 5, verbose = 1, n_jobs=-1,)
mean = accuracies.mean()
variance = accuracies.std()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  1.9min remaining:  2.9min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.0min finished


In [16]:
print(mean)
print(variance)

0.8446249961853027
0.018373717912085523


In [17]:
#Dealing with Overfitting
#NOTE: Out current model is not overfitted hence it is not required to deal with overfitting here. But this step is needed in case of overfitting

#Dropout regularization
#Importing the Keras Libraries & Packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout #This would disable the neuron in each layer and this would make our model to learn more independently.

#Initialize the ANN
classifier = Sequential()

#Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform',activation = 'relu', input_shape = (11,)))
classifier.add(Dropout(p = 0.1)) #value range 0-1, p is the proportion of neuron to disable. If there are 10 neurons in the layer and we are we give p=0.1 that means we disable 1 neuron

#Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform',activation = 'relu'))
classifier.add(Dropout(p = 0.1))

#Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform',activation = 'sigmoid'))

#Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics= ['accuracy'] )

#Fitting the ANN to the training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)



C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  from ipykernel import kernelapp as app
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`


Epoch 1/100
8000/8000 [==============================] - 3s 333us/step - loss: 0.4884 - accuracy: 0.7955
Epoch 2/100
8000/8000 [==============================] - 2s 231us/step - loss: 0.4260 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 2s 231us/step - loss: 0.4205 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 2s 228us/step - loss: 0.4147 - accuracy: 0.8094
Epoch 5/100
8000/8000 [==============================] - 2s 234us/step - loss: 0.4099 - accuracy: 0.8234
Epoch 6/100
8000/8000 [==============================] - 2s 232us/step - loss: 0.4066 - accuracy: 0.8223
Epoch 7/100
8000/8000 [==============================] - 2s 230us/step - loss: 0.4046 - accuracy: 0.8226
Epoch 8/100
8000/8000 [==============================] - 2s 239us/step - loss: 0.4062 - accuracy: 0.8201
Epoch 9/100
8000/8000 [==============================] - 2s 232us/step - loss: 0.4045 - accuracy: 0.8275
Epoch 10/100
8000/8000 [==============================]

8000/8000 [==============================] - 4s 496us/step - loss: 0.4023 - accuracy: 0.8403
Epoch 79/100
8000/8000 [==============================] - 4s 494us/step - loss: 0.4009 - accuracy: 0.8380
Epoch 80/100
8000/8000 [==============================] - 4s 464us/step - loss: 0.4009 - accuracy: 0.8394
Epoch 81/100
8000/8000 [==============================] - 3s 342us/step - loss: 0.4039 - accuracy: 0.8356
Epoch 82/100
8000/8000 [==============================] - 3s 392us/step - loss: 0.4010 - accuracy: 0.8361
Epoch 83/100
8000/8000 [==============================] - 4s 469us/step - loss: 0.4009 - accuracy: 0.8384
Epoch 84/100
8000/8000 [==============================] - 3s 325us/step - loss: 0.4006 - accuracy: 0.8385
Epoch 85/100
8000/8000 [==============================] - 2s 273us/step - loss: 0.4029 - accuracy: 0.8378
Epoch 86/100
8000/8000 [==============================] - 2s 238us/step - loss: 0.4006 - accuracy: 0.8380
Epoch 87/100
8000/8000 [==============================] - 2

## Hyper Parameter Tunning using Gridsearch

In [18]:
# Importing the libraries

import numpy as np
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

# Encoding categorical data
# Encoding the Independent Variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

labelencoder_X_1 = LabelEncoder()
X[:,2] = labelencoder_X_1.fit_transform(X[:,2])

#labelencoder_X_2 = LabelEncoder()
#X[:,2] = labelencoder_X_2.fit_transform(X[:,2])

ct = ColumnTransformer([("Geography", OneHotEncoder(),[1])], remainder="passthrough")
X = np.array(ct.fit_transform(X), dtype=np.int64)
#avoiding dummy trap:
X = X[:, 1:]

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [25]:
#Doing Grid Search:
# Evaluating, Improving and Tuning the ANN
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [26]:
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [25, 32],
              'epochs': [100, 500],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10, n_jobs= -1)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Epoch 1/500
8000/8000 [==============================] - 0s 48us/step - loss: 0.5744 - accuracy: 0.7958
Epoch 2/500
8000/8000 [==============================] - 0s 23us/step - loss: 0.4446 - accuracy: 0.7960
Epoch 3/500
8000/8000 [==============================] - 0s 23us/step - loss: 0.4341 - accuracy: 0.7960
Epoch 4/500
8000/8000 [==============================] - 0s 24us/step - loss: 0.4301 - accuracy: 0.7960
Epoch 5/500
8000/8000 [==============================] - 0s 22us/step - loss: 0.4274 - accuracy: 0.7960
Epoch 6/500
8000/8000 [==============================] - 0s 23us/step - loss: 0.4249 - accuracy: 0.7960
Epoch 7/500
8000/8000 [==============================] - 0s 23us/step - loss: 0.4223 - accuracy: 0.8065
Epoch 8/500
8000/8000 [==============================] - 0s 23us/step - loss: 0.4204 - accuracy: 0.8200
Epoch 9/500
8000/8000 [==============================] - 0s 24us/step - loss: 0.4187 - accuracy: 0.8207
Epoch 10/500
8000/8000 [==============================] - 0s 23u

8000/8000 [==============================] - 0s 23us/step - loss: 0.4009 - accuracy: 0.8351
Epoch 80/500
8000/8000 [==============================] - 0s 23us/step - loss: 0.4009 - accuracy: 0.8346
Epoch 81/500
8000/8000 [==============================] - 0s 23us/step - loss: 0.4011 - accuracy: 0.8345
Epoch 82/500
8000/8000 [==============================] - 0s 23us/step - loss: 0.4009 - accuracy: 0.8359
Epoch 83/500
8000/8000 [==============================] - 0s 24us/step - loss: 0.4010 - accuracy: 0.8347
Epoch 84/500
8000/8000 [==============================] - 0s 23us/step - loss: 0.4008 - accuracy: 0.8350
Epoch 85/500
8000/8000 [==============================] - 0s 21us/step - loss: 0.4009 - accuracy: 0.8357
Epoch 86/500
8000/8000 [==============================] - 0s 23us/step - loss: 0.4010 - accuracy: 0.8351
Epoch 87/500
8000/8000 [==============================] - 0s 23us/step - loss: 0.4006 - accuracy: 0.8349
Epoch 88/500
8000/8000 [==============================] - 0s 25us/st

8000/8000 [==============================] - 0s 23us/step - loss: 0.3996 - accuracy: 0.8349
Epoch 157/500
8000/8000 [==============================] - 0s 26us/step - loss: 0.3998 - accuracy: 0.8360
Epoch 158/500
8000/8000 [==============================] - 0s 23us/step - loss: 0.3999 - accuracy: 0.8344
Epoch 159/500
8000/8000 [==============================] - 0s 23us/step - loss: 0.3997 - accuracy: 0.8359
Epoch 160/500
8000/8000 [==============================] - 0s 25us/step - loss: 0.3995 - accuracy: 0.8356
Epoch 161/500
8000/8000 [==============================] - 0s 23us/step - loss: 0.3995 - accuracy: 0.8363
Epoch 162/500
8000/8000 [==============================] - 0s 25us/step - loss: 0.3996 - accuracy: 0.8353
Epoch 163/500
8000/8000 [==============================] - 0s 24us/step - loss: 0.3998 - accuracy: 0.8354 0s - loss: 0.4026 - accuracy: 0.83
Epoch 164/500
8000/8000 [==============================] - 0s 23us/step - loss: 0.3998 - accuracy: 0.8355
Epoch 165/500
8000/8000 [

8000/8000 [==============================] - 0s 21us/step - loss: 0.3994 - accuracy: 0.8354
Epoch 233/500
8000/8000 [==============================] - 0s 23us/step - loss: 0.3994 - accuracy: 0.8355
Epoch 234/500
8000/8000 [==============================] - 0s 23us/step - loss: 0.3990 - accuracy: 0.8363
Epoch 235/500
8000/8000 [==============================] - 0s 25us/step - loss: 0.3994 - accuracy: 0.8366
Epoch 236/500
8000/8000 [==============================] - 0s 22us/step - loss: 0.3995 - accuracy: 0.8349
Epoch 237/500
8000/8000 [==============================] - 0s 23us/step - loss: 0.3991 - accuracy: 0.8339
Epoch 238/500
8000/8000 [==============================] - 0s 23us/step - loss: 0.3995 - accuracy: 0.8370
Epoch 239/500
8000/8000 [==============================] - 0s 23us/step - loss: 0.3992 - accuracy: 0.8354
Epoch 240/500
8000/8000 [==============================] - 0s 24us/step - loss: 0.3994 - accuracy: 0.8359
Epoch 241/500
8000/8000 [==============================] - 0

8000/8000 [==============================] - 4137s 517ms/step - loss: 0.3993 - accuracy: 0.8351
Epoch 310/500
8000/8000 [==============================] - 0s 28us/step - loss: 0.3992 - accuracy: 0.8356
Epoch 311/500
8000/8000 [==============================] - 0s 45us/step - loss: 0.3995 - accuracy: 0.8349
Epoch 312/500
8000/8000 [==============================] - 1s 70us/step - loss: 0.3993 - accuracy: 0.8342
Epoch 313/500
8000/8000 [==============================] - 1s 65us/step - loss: 0.3995 - accuracy: 0.8344
Epoch 314/500
8000/8000 [==============================] - 0s 30us/step - loss: 0.3989 - accuracy: 0.8379
Epoch 315/500
8000/8000 [==============================] - 0s 31us/step - loss: 0.3993 - accuracy: 0.8365
Epoch 316/500
8000/8000 [==============================] - 0s 29us/step - loss: 0.3994 - accuracy: 0.8354
Epoch 317/500
8000/8000 [==============================] - 0s 29us/step - loss: 0.3992 - accuracy: 0.8347
Epoch 318/500
8000/8000 [==============================]

8000/8000 [==============================] - 0s 40us/step - loss: 0.3990 - accuracy: 0.8359
Epoch 387/500
8000/8000 [==============================] - 0s 31us/step - loss: 0.3988 - accuracy: 0.8359
Epoch 388/500
8000/8000 [==============================] - 0s 30us/step - loss: 0.3992 - accuracy: 0.8350
Epoch 389/500
8000/8000 [==============================] - 0s 29us/step - loss: 0.3989 - accuracy: 0.8357
Epoch 390/500
8000/8000 [==============================] - 0s 32us/step - loss: 0.3990 - accuracy: 0.8360
Epoch 391/500
8000/8000 [==============================] - 0s 29us/step - loss: 0.3990 - accuracy: 0.8355
Epoch 392/500
8000/8000 [==============================] - 0s 29us/step - loss: 0.3988 - accuracy: 0.8349
Epoch 393/500
8000/8000 [==============================] - 0s 29us/step - loss: 0.3991 - accuracy: 0.8367
Epoch 394/500
8000/8000 [==============================] - 0s 28us/step - loss: 0.3988 - accuracy: 0.8360
Epoch 395/500
8000/8000 [==============================] - 0

8000/8000 [==============================] - 0s 25us/step - loss: 0.3986 - accuracy: 0.8359
Epoch 464/500
8000/8000 [==============================] - 0s 26us/step - loss: 0.3988 - accuracy: 0.8336 0s - loss: 0.3985 - accuracy: 
Epoch 465/500
8000/8000 [==============================] - 0s 27us/step - loss: 0.3985 - accuracy: 0.8351
Epoch 466/500
8000/8000 [==============================] - 0s 24us/step - loss: 0.3987 - accuracy: 0.8350
Epoch 467/500
8000/8000 [==============================] - 0s 24us/step - loss: 0.3990 - accuracy: 0.8355
Epoch 468/500
8000/8000 [==============================] - 0s 24us/step - loss: 0.3984 - accuracy: 0.8361
Epoch 469/500
8000/8000 [==============================] - 0s 24us/step - loss: 0.3986 - accuracy: 0.8342
Epoch 470/500
8000/8000 [==============================] - 0s 25us/step - loss: 0.3989 - accuracy: 0.8344
Epoch 471/500
8000/8000 [==============================] - 0s 25us/step - loss: 0.3984 - accuracy: 0.8356
Epoch 472/500
8000/8000 [====

In [27]:
print(best_accuracy)

0.851625


In [28]:
print(best_parameters)

{'batch_size': 32, 'epochs': 500, 'optimizer': 'rmsprop'}
